<a href="https://colab.research.google.com/github/JHyunjun/Data-Pre-Processing/blob/main/TimeSeries_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun,JANG
# limitsinx.tistory.com
# Last revision date : 2023.04.29

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, GRU, Dropout, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D


xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883
xy_train = np.delete(xy_train ,3, axis = 1)
xy_test = np.delete(xy_test ,3, axis = 1)

start_test = 1000
middle_test = 2000

middle_test1 = 3000
end_test = 10000

xy_test_1 = xy_test[:start_test,:]
xy_test_1n2 = xy_test[start_test:middle_test,:]
xy_test_2 = xy_test[middle_test1:end_test,:]

xy_train = np.concatenate((xy_train, xy_test_1n2), axis = 0)
xy_test = np.concatenate((xy_test_1, xy_test_2), axis = 0)

def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

def origin_minmax(data) : # HJ STYLE
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])

  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max_train, arr_min_train = origin_minmax(xy_train)
arr_max_test, arr_min_test = origin_minmax(xy_test)
arr_max_total = arr_max_train
arr_min_total = arr_min_train
print(arr_max_train, arr_min_train)
print(arr_max_test, arr_min_test)

for i in range(xy_train.shape[1]) : 
  print(i)
  if(arr_max_train[i] > arr_max_test[i]) : 
    arr_max_total[i] = arr_max_train[i]
  else : 
    arr_max_total[i] = arr_max_test[i]
  pass

for i in range(xy_train.shape[1]) : 
  if(arr_min_train[i] < arr_min_test[i]) : 
    arr_min_total[i] = arr_min_train[i]
  else : 
    arr_min_total[i] = arr_min_test[i]
  pass

print(arr_max_total, arr_min_total)

def HJ_MinMaxScaler(data) : 
  for i in range(data.shape[1]) : 
    numerator = data[:,i] - arr_min_total[i]
    denominator = arr_max_total[i] - arr_min_total[i]
    data[:,i] = numerator / (denominator + 1e-7)
    
  return data

def HJ_backMinMax(data) : 
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (arr_max_total[i] - arr_min_total[i]) + arr_min_total[i]

  return data

xy_train_scale = HJ_MinMaxScaler(xy_train)
xy_test_scale = HJ_MinMaxScaler(xy_test)


In [ ]:
print(xy_train.shape)
print(xy_test.shape)

In [ ]:
def reshape_timeseries_to_image(data, window_size):
    num_features = data.shape[1]
    num_samples = data.shape[0] - window_size + 1
    reshaped_data = np.zeros((num_samples, window_size, num_features))

    for i in range(num_samples):
        reshaped_data[i] = data[i:i+window_size]

    return reshaped_data

In [ ]:
TIME_STEPS = 5

train_image = reshape_timeseries_to_image(xy_train, TIME_STEPS)
test_image = reshape_timeseries_to_image(xy_test, TIME_STEPS)
print(train_image.shape,train_image[0,:,:])
print(test_image.shape)